In [13]:
import pandas as pd
import numpy as np

claims = pd.read_csv("../../data/polish-fake-news-dataset/claims.csv")
claims

,id,content,source,label,tags,publication_date,author,link
0,1,Masowe zwolnienia w marcu tego roku objęły 17 ...,demagog_org,Prawda,"Gospodarka,UE,Praca,Bezrobocie",1717168920,Anna Bryłka,https://demagog.org.pl/wypowiedzi/ilu-pracowni...
1,2,"W proponowanych zmianach traktatowych, które p...",demagog_org,Fałsz,"UE,Uchodźcy",1717158300,Dominik Tarczyński,https://demagog.org.pl/wypowiedzi/obozy-na-pol...
2,3,Polacy wybierają jedną z największych ilości w...,demagog_org,Prawda,"UE,Wybory europarlamentarne",1717156380,Patryk Jaki,https://demagog.org.pl/wypowiedzi/eurowybory-i...
3,4,W tej chwili np. w Warszawie ma powstać kolejn...,demagog_org,Prawda,"Religia,Austria",1716996000,Krzysztof Bosak,https://demagog.org.pl/wypowiedzi/islam-a-aust...
4,5,"Pracujemy najwięcej jako Polacy w Europie, a n...",demagog_org,W większości prawda,"Praca,Gospodarka,UE,Wybory,Wybory europarlamen...",1716981540,Robert Biedroń,https://demagog.org.pl/wypowiedzi/zarabiamy-po...
...,...,...,...,...,...,...,...,...
5769,5771,Komisarz Virginijus Sinkevičius skieruje spraw...,fake_news_pl,Fałsz,srodowisko,1600116892,NaN,https://fakenews.pl/srodowisko/nie-sprawa-mier...
5770,5772,Zdjęcia pokazujące śnięte ryby pochodzą z okol...,fake_news_pl,W większości prawda,srodowisko,1599569432,NaN,https://fakenews.pl/srodowisko/sniete-ryby-w-w...
5771,5773,Brązowy kolor Wisły to skutek zrzutu ścieków z...,fake_news_pl,Fałsz,srodowisko,1599220215,NaN,https://fakenews.pl/srodowisko/scieki-plynace-...
5772,5774,Samoloty rozpylają szkodliwe substancje (takie...,fake_news_pl,Fałsz,srodowisko,1590154590,NaN,https://fakenews.pl/srodowisko/nie-w-polsce-ni...


In [21]:
claims["tags"] = claims["tags"].fillna("")

unique_tags = (claims["tags"].str.split(",")
               .explode()
               .str.strip()
               .unique())
unique_tags

array(['Gospodarka', 'UE', 'Praca', 'Bezrobocie', 'Uchodźcy',
       'Wybory europarlamentarne', 'Religia', 'Austria', 'Wybory',
       'Ukraina', 'Migracja', 'Niemcy', 'Transport',
       'Sprawy międzynarodowe', 'Bezpieczeństwo', 'Ekologia', 'Klimat',
       'Równouprawnienie', 'Zwierzęta', 'Zdrowie', 'Chiny', 'Energetyka',
       'LGBTQIA', 'Włochy', 'Wymiar sprawiedliwości', 'Służby specjalne',
       'Polityka krajowa', 'Rolnictwo', 'Technologia', 'Sądownictwo',
       'Wojsko', 'NATO', 'USA', 'Finanse', 'Budżet', 'Policja', 'Podatki',
       'Białoruś', 'Górnictwo', 'Rosja', 'Edukacja', 'Szkolnictwo wyższe',
       'Polityka społeczna', 'Demografia', 'Sztuczna inteligencja',
       'Referendum', 'Aborcja', 'Infrastruktura', 'Samorząd',
       'Wybory samorządowe', 'Wrocław', 'Mieszkalnictwo', 'Rzeszów',
       'Kraków', 'Turystyka', 'Sejm', 'Konstytucja', 'Warszawa', 'Gdańsk',
       'Historia', 'Lublin', 'Gdynia', 'Budownictwo', 'Łódź', 'Katowice',
       'Kultura', 'Smog', 'Ośw

In [24]:
import spacy
from spacy.language import Language
from spacy.tokens import Doc
from transformers import pipeline

sentiment_pipeline = pipeline("sentiment-analysis", model="nie3e/sentiment-polish-gpt2-large")

if not Doc.has_extension("sentiment"):
    Doc.set_extension("sentiment", default=None)
if not Doc.has_extension("score"):
    Doc.set_extension("score", default=None)

@Language.component("sentiment_component")
def sentiment_component(doc):
    result = sentiment_pipeline(doc.text)[0]
    doc._.sentiment = result["label"].lower()
    doc._.score = result["score"]
    return doc

nlp = spacy.blank("pl")
nlp.add_pipe("sentiment_component", last=True)

# Test
doc = nlp("@pisorgpl Pis skazuje obecnych 30 latków na emerytury głodowe, które już za twoich czasów Morawiecki, wzrosły 3 krotnie,  a to dopiero początek!")
print(f"Sentyment: {doc._.sentiment}, Pewność: {doc._.score:.2f}")


Device set to use mps:0


Sentyment: negative, Pewność: 0.99
